<a href="https://colab.research.google.com/github/dooooing/2023-ESAA-/blob/main/0922_%EC%A0%9C%EC%B6%9C_%ED%95%B8%EC%A6%88%EC%98%A8_7%EC%9E%A5_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_p_245_271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7. 앙상블 학습과 랜덤 포레스트

**앙상블 학습** : 일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측 얻을 수 있다

ex) 랜덤 포레스트 : 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정트리 분류기 훈련 -> 모든 개별 트리의 예측 구해 가장 많은 선택을 받은 클래스를 예측으로 삼아

## 7.1 투표 기반 분류기


**직접 투표(하드 보팅) 분류기** : 각 분류기의 예측을 모아서 가장 많이 선택된 클래스 예측.(다수결 투표)

* 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도 높을 경우

* 각 분류기가 약한 학습기일지라도 충분하게 많고 다양하다면 앙상블은 강한 학습기 가능

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y=make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [2]:
# 여러 분류기 조합해 사이킷런의 VotingClassifier 만들고 훈련
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


**간접 투표(소프트 보팅)** : 모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_proba()메서드 있으면), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스 예측

* voting="soft"로 사용
* SVC는 probability 매개변수를 True로 지정

## 7.2 배깅과 페이스팅

**배깅** : 훈련 세트에서 중복을 허용하여 샘플링하는 방식

**페이스팅** : 중복을 허용하지 않고 샘플링하는 방식

* 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용 가능
* 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링 가능

* 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측 만들어
  + 수집함수는 분류일 때는 통계적 최빈값, 회귀일 때는 평균 계산
  + 수집함수 통과하면 일반적으로 앙상블의 결과 -> 편향은 비슷. 분산은 줄어들어.

### 7.2.1 사이킷런의 배깅과 페이스팅

사이킷런은 **BaggingClassifier(BaggingRegressor)** 제공. 페이스팅 사용하려면 **bootstrap=False**로 지정

* n_jobs 매개변수 : 사이킷런이 훈련과 예측에 사용할 CPU 코어 수 지정
  + -1로 지정하면 가용한 모든 코어 사용

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

* BaggingClassifier는 기반이 되는 분류기가 클래스 확률을 추정할 수 있으면(predict_proba() 함수 있으면) 직접 투표 대신 자동으로 간접 투표 방식 사용

* 앙상블은 비슷한 편향에서 더 작은 분산 만들어 -> 훈련 세트의 오차 수 거의 비슷하지만 결정 경계는 덜 불규칙

* 부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성 증가 -> 배깅이 페이스팅보다 편향이 조금 더 높지만 분산 작아
* 전반적으로 배깅을 더 선호. 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅 모두 평가해서 더 나은 쪽 선택.

### 7.2.2 oob 평가

배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다

평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다. 선택되지 않은 훈련 샘플의 나머지 37%를 **oob 샘플**

* 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻는다

* 사이킷런에서 BaggingClassifier 만들 때 **oob_score=True** 지정하면 훈련 끝난 후 자동으로 oob 평가 수행
  + 평가 점수 결과는 **oob_score_** 변수에 저장

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9375

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

* oob 샘플에 대한 결정 함수의 값은 **oob_decision_function_** 변수에서 확인 가능
  + 결정 함수는 각 훈련 샘플의 클래스 확률 반환

In [7]:
bag_clf.oob_decision_function_

array([[0.99516908, 0.00483092],
       [0.99428571, 0.00571429],
       [0.74611399, 0.25388601],
       [1.        , 0.        ],
       [0.01015228, 0.98984772],
       [0.88557214, 0.11442786],
       [0.01744186, 0.98255814],
       [0.78191489, 0.21808511],
       [0.75816993, 0.24183007],
       [0.01612903, 0.98387097],
       [0.01176471, 0.98823529],
       [0.2565445 , 0.7434555 ],
       [0.80788177, 0.19211823],
       [0.08585859, 0.91414141],
       [0.01136364, 0.98863636],
       [0.99473684, 0.00526316],
       [0.        , 1.        ],
       [0.70157068, 0.29842932],
       [0.32960894, 0.67039106],
       [0.        , 1.        ],
       [0.48044693, 0.51955307],
       [0.21827411, 0.78172589],
       [0.        , 1.        ],
       [0.28735632, 0.71264368],
       [0.53888889, 0.46111111],
       [1.        , 0.        ],
       [0.06179775, 0.93820225],
       [0.23121387, 0.76878613],
       [0.04519774, 0.95480226],
       [1.        , 0.        ],
       [1.

## 7.3 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성 샘플링도 지원 -> **max_features, bootstrap_features** 두 매개변수로 조절

* 샘플이 아니고 특성에 대한 샘플링 -> 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련돼

* 매우 고차원의 데이터셋 다룰 때 유용

**랜덤 패치 방식** : 훈련 특성과 샘플을 모두 샘플링하는 것

**랜덤 서브스페이스 방식** : 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것

* 특성 샘플링은 더 다양한 예측기 만들며 편향 늘리는 대신 분산 낮춰

## 7.4 랜덤 포레스트

**랜덤 포레스트** : 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블. **RandomForestClassifier. RandomForestRegressor** 사용

* 전형적으로 max_samples를 훈련 세트의 크기로 지정


In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

트리 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성 더 주입

-> 트리 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### 7.4.1 엑스트라 트리

랜덤 포레스트에서 트리 만들 때 각 노드는 무작위로 특성의 서브셋 만들어 분할에 사용 -> 더욱 무작위하게 만들기 위해 최적의 임곗값 찾는 대신 후보 특성 사용해 무작위 분할 후 최상의 분할 선택

**익스트림 랜덤 트리(엑스트라 트리)** : 극단적으로 무작위한 트리의 랜덤 포레스트. 사이킷런의 **ExtraTreesClassifier** 사용

* 편향 늘어나지만 대신 분산 낮추게 돼
* 일반적으로 랜덤 포레스트보다 엑스트라 트리가 훨씬 빨라

### 7.4.2 특성 중요도

랜덤 포레스트의 장점 : 특성의 상대적 중요도를 측정하기 쉽다는 것

사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도 측정

* 사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값 정규화. 이 값은 **feature_importances_** 변수에 저장

In [10]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09628302580463333
sepal width (cm) 0.02661649284107417
petal length (cm) 0.43466149594270886
petal width (cm) 0.4424389854115837


## 7.5 부스팅

**부스팅** : 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

* 에이다부스트, 그레이디언트 부스팅

### 7.5.1 에이다부스트

**에이다부스트** : 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높여 이전 예측기를 보완하는 새로운 예측기를 만드는 방법

ex) 먼저 알고리즘이 기반이 되는 첫 번째 분류기를 훈련 세트에서 훈련시키고 예측 만들어 -> 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높여 -> 두 번째 분류기는 업데이트된 가중치 사용해 훈련 세트에서 훈련하고 다시 예측 만들어 -> 다시 가중치 업데이트하는 식으로 계속

=> 경사 하강법과 비슷한 면. 에이다부스트는 점차 더 좋아지도록 앙상블에 예측기 추가.

* 배깅이나 페이스팅과 비슷한 방식으로 에측 만들어. but 가중치가 적용된 훈련 세트의 전반적인 정확도에 따라 예측기마다 다른 가중치 적용. 예측기가 정확할수록 가중치가 더 높아지게 돼.
* 샘플의 가중치 업데이트 -> 모든 샘플의 가중치 정규화 -> 새 예측기가 업데이트된 가중치 사용해 훈련되고 전체 과정이 반복 -> 지정된 에측기 수에 도달하거나 완벽한 예측기가 만들어지면 중지.


* 사이킷런은 **SAMME**라는 에이다부스트의 다중 클래스 버전 사용
  + 예측기가 클래스의 확률을 추정할 수 있다면(predict_proba() 메서드가 있다면) 사이킷런은 **SAMME.R**이라는 변종 사용
  + 예측값 대신 클래스 확률에 기반하며 일반적으로 성능이 더 좋다

* 사이킷런의 **AdaBoostClassifier** 사용하여 200개의 아주 얕은 결정 트리를 기반으로 하는 에이다부스트 분류기 훈련


In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅

**그레이디언트 부스팅** : 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가. 이전 예측기가 만든 잔여 오차에 새로운 예측기 학습

In [12]:
# DecisionTreeRegressor를 훈련 세트에 학습
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [13]:
# 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor 훈련
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [14]:
# 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델 훈련
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [15]:
# 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측 더하면 돼
# y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

* 사이킷런의 **GradientBoostingRegressor** 사용해 GBRT 앙상블 훈련
  + 매개변수 max_depth, min_samples_leaf

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

* **learning_rate** 매개변수 : 각 트리의 기여 정도 조절
  + 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리 필요하지만 일반적으로 예측 성능은 좋아져 : 축소 라는 규제 방법

* 최적의 트리 수를 찾기 위해서는  조기 종료 기법 사용 -> **staged_predict()** 메서드
  + 훈련의 각 단계에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자 반환

In [17]:
# 120개의 트리로 GBRT 앙상블 훈련 -> 최적의 트리 수 찾기 위해 각 훈련 단게에서 검증 오차 측정 -> 새로운 GBRT 앙상블 훈련
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=104)

* 실제로 훈련 중지하는 방법으로 조기 종료 구현 -> **warm_start = True**로 설정
  + 사이킷런이 fit() 메서드가 호출될 때 기존 트리 유지하고 훈련 추가할 수 있도록 해줘

In [18]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break    # 조기 종료

* 각 트리가 훈련할 때 사용할 훈련 샘플의 비율 지정할 수 있는 **subsample** 매개변수 지원
  + 편향이 높아지는 대신 분산 낮아져. 훈련 속도 상당히 높여
  + 이 기법을 **확률적 그레이디언트 부스팅**

* 최적화된 그레이디언트 부스팅 구현으로 **XGBoost** 파이썬 라이브러리 유명

In [19]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [20]:
# 자동 조기 종료와 같은 여러 좋은 기능도 제공
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.39493
[1]	validation_0-rmse:0.33072
[2]	validation_0-rmse:0.29555
[3]	validation_0-rmse:0.27883
[4]	validation_0-rmse:0.27235
[5]	validation_0-rmse:0.27094
[6]	validation_0-rmse:0.26597
[7]	validation_0-rmse:0.26318
[8]	validation_0-rmse:0.26235
[9]	validation_0-rmse:0.26154
[10]	validation_0-rmse:0.26136
[11]	validation_0-rmse:0.26149
[12]	validation_0-rmse:0.26119
[13]	validation_0-rmse:0.26100
[14]	validation_0-rmse:0.26087
[15]	validation_0-rmse:0.26078
[16]	validation_0-rmse:0.26080
[17]	validation_0-rmse:0.26075
[18]	validation_0-rmse:0.26072
[19]	validation_0-rmse:0.26071
[20]	validation_0-rmse:0.26069
[21]	validation_0-rmse:0.26070
[22]	validation_0-rmse:0.26069
[23]	validation_0-rmse:0.26069
[24]	validation_0-rmse:0.26069
[25]	validation_0-rmse:0.26065
[26]	validation_0-rmse:0.26048
[27]	validation_0-rmse:0.26034
[28]	validation_0-rmse:0.26022
[29]	validation_0-rmse:0.26011
[30]	validation_0-rmse:0.26002
[31]	validation_0-rmse:0.25995
[32]	validation_0-

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


## 7.6 스태킹

**스태킹** : '앙상블에 속한 모든 예측기의 예측을 수합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까요?'라는 기본 아이디어로 출발

* 각각 다른 값을 예측하고 마지막 예측기(블렌더 또는 메타 학습기)가 예측을 입력으로 받아 최종 예측 만들어

* 블렌더를 학습시키는 일반적인 방법은 홀드 아웃 세트 사용하는 것

훈련 세트를 두 개의 서브셋으로 나눈다 -> 첫 번째 서브셋은 첫 번째 레이어의 예측을 훈련시키기 위해 사용. 첫 번째 레이어의 예측기를 사용해 두 번째 세트에 대한 예측 만들어. -> 타깃값을 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있다. 블렌더가 새 훈련 세트로 훈련돼. 즉, 첫 번째 레이어의 예측을 가지고 타깃값을 예측하도록 학습된다

* 이런 방식의 블렌더를 여러 개 훈련시키는 것도 가능. 블렌더만의 레이어

훈련 세트를 세 개의 서브셋으로 나눈다 -> 첫 번째 세트는 첫 번째 레이어를 훈련시키는 데 사용되고 나머지 세트도 각 레이어를 훈련시키는 데 사용돼 -> 각 레이어를 차례대로 실행해서 새로운 샘플에 대한 예측 만들 수 있다